# SPATIAL INTERPOLATION

This Jupyter Notebook can be used to play around with the interpolation script that we will experiment on the streamed data in the previous step. We will perform two kinds of streaming implementation

<!-- - Event Detection : In this example we will check the average of  -->

In [1]:
import pandas as pd
import json
import sys
import time
import socket
from confluent_kafka import Consumer, KafkaError, KafkaException

In [2]:
df = pd.DataFrame()

## Set topic name as set in sendStream.py
topic = "pm25_stream"

In [5]:
conf = {'bootstrap.servers': '0.0.0.0:9092',
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'group.id': socket.gethostname()}
### EMD: AVOID MAKING CHANGES ###

consumer = Consumer(conf)

running = True
consumer.subscribe([topic])

%3|1661972946.132|FAIL|rdkafka#consumer-3| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1661972947.131|FAIL|rdkafka#consumer-3| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)


In [ ]:
## Import Libraries

import geopandas as gpd
import pandas as pd
import json
import numpy as np
from functools import partial
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata
import geojson

In [ ]:
## Read the output of the streamed file

df = pd.read_csv('../data/streamed_output.csv').drop(['Unnamed: 0'], axis=1)
df.head()

In the above dataframe, we have data for three different days for each of the 10 locations

In [ ]:
##
df['day'].value_counts()

In [ ]:
# Convert Pandas to GeoPandas
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.set_crs(epsg=4326, inplace=True)
gdf.to_crs(epsg=3035, inplace=True)
gdf.drop(['lon','lat'], axis=1, inplace=True)
gdf.head()

Plot The Values

In [ ]:
gdf.plot()

The library that we use for interpolation does not directly allow interpolation to a Polygon or is a rather complicated approach. To coutner this, a shapefile "area.geojson" is provided that provides a simpler rectangular extent around Germany.

We first interpolate to this rectangular extent, finally we will crop the boundary of Germany and get a more realistic output

In [ ]:
## Read the rectangular extent

with open('data/area.geojson') as f:
    gj = geojson.load(f)
    
geometry = gj['features'][0]['geometry']
geometry

In [ ]:
## Plot Germany
germany = gpd.read_file('data/germany_simplified.shp')
germany.to_crs(epsg=3035, inplace=True)
germany.plot()

In [ ]:
## Plot Rectangular Extent
gpd.read_file('data/area.geojson').plot()

## Interpolation

In this course, we will use Geocube to generate an interpolated surface.
Note: You can use more sophisticated Libraries as well like "PyKrige"

In [ ]:
geo_grid_cubic = make_geocube(
    gdf,
    measurements=["value"],
    rasterize_function=partial(rasterize_points_griddata, method="linear", all_touched=True),
    interpolate_na_method="nearest",
    geom=geometry,
    resolution = (-1500,1500) ## Lowering this number will take longer processing time and more memory
)

In [ ]:
## Plot the Interpolation
geo_grid_cubic.value.plot.imshow()

In [ ]:
## Convert xarray (Geocube) output to a DataFrame

output = geo_grid_cubic.to_dataframe().reset_index()
output.head()

In [ ]:
## Convert this DataFrame to Geopandas

gdf_interpolated = gpd.GeoDataFrame(output, geometry=gpd.points_from_xy(output.x, output.y)).set_crs(epsg=3035)
gdf_interpolated['geometry'] = gdf_interpolated.geometry.buffer(0.0001)
gdf_interpolated.head()

In [ ]:
gdf_interpolated.shape

Now we will perform a spatial join, only the region that intersects with the German shapefile will be retained

In [ ]:
cropped = germany.sjoin(gdf_interpolated, how="left")[['geometry','value']]
cropped.shape

Save Results

In [ ]:
# cropped.plot('value', cmap='OrRd') 
## To plot the results, it is recommended to reduce the resolution further down 
## as it uses a lot of memory and might crash your notebook

In [ ]:
## Export results to a shapefile (More Time Consuming)
# cropped.to_file('data/interpolated_cropped.shp', driver='ESRI Shapefile')

In [ ]:
## Export the rectangular interpolation as raster, quicker to process
geo_grid_cubic.rio.to_raster("data/interpolated_rectangular.tif")

#### END
